<a href="https://colab.research.google.com/github/Arvinth-s/MIMO/blob/main/BF_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#OM NAMO NARAYANA

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive/')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive/


In [4]:
os.listdir('/content/drive/MyDrive/data sets')

['20db', '0db', '-20db', 'Lest2', 'Lest1']

In [6]:
os.listdir('/content/drive/MyDrive/data sets/20db/train')

['ecsi.mat', 'pcsi.mat']

In [10]:
target_dir = '/content/drive/MyDrive/data sets/'

In [ ]:
import torch.nn as nn
import torch.nn.functional as  F
from torch import optim
import torch

In [8]:
import random
import math
import cmath
import scipy.io as sio

In [ ]:

H = sio.loadmat(target_dir + 'BF_Design/train/pcsi.mat')
H_est = sio.loadmat(target_dir + 'BF_Design/train/ecsi.mat')
print(H)

In [47]:
H_20 = []
H_20 = sio.loadmat(target_dir + '20db/train/pcsi.mat')
H_20_est = sio.loadmat(target_dir + '20db/train/ecsi.mat')
print(H_20)
print(H_20['pcsi'].shape)
print(H_20_est)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Apr 20 13:04:58 2019', '__version__': '1.0', '__globals__': [], 'pcsi': array([[[ 1.50147694+0.44686848j,  1.20796114-0.96018607j,
         -0.11359252-1.463618j  , ...,  0.42528519-0.58007624j,
         -0.25945843-0.77886998j, -0.84394454-0.29822516j]],

       [[ 0.38892007-0.32636459j,  0.29823891+0.2183917j ,
          0.66302605+0.63223397j, ...,  1.15716358+0.06924648j,
          0.71931132+0.16306212j,  0.59819429+0.53650335j]],

       [[ 0.83594924-0.44808131j,  0.98906444-1.006238j  ,
          0.54324663-1.05449685j, ..., -0.53927657+0.08589144j,
         -0.60645661+0.96209394j, -0.34886921+1.23299291j]],

       ...,

       [[ 0.52511091-0.15074378j,  0.24452902+0.69684149j,
         -0.66109416+0.94840944j, ...,  0.67964394+0.11438444j,
         -0.07755882+0.76458209j, -1.06284711+0.47990537j]],

       [[ 1.1192355 +0.29365041j,  0.77098107+0.50310646j,
          0.62200443+0.52450842j, ..., -0.9

In [29]:
import numpy as np
# from utils import *
from tensorflow.python.keras.layers import *
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as k

In [56]:
H_20input = np.expand_dims(np.concatenate([np.real(H_20_est['ecsi']), np.imag(H_20_est['ecsi'])], 1), 1)
# H_20 = np.squeeze(H_20['pcsi'])
SNR = np.power(10, np.random.randint(-20, 20, [H_20.shape[0], 1]) / 10)
print(H_20input.shape)
print(H_20.shape)

(100000, 1, 2, 64)
(100000, 64)


In [30]:
Nt = 64  # the number of antennas
P = 1   # the normalized transmit power


def trans_Vrf(temp):
    v_real = tf.cos(temp)
    v_imag = tf.sin(temp)
    vrf = tf.cast(tf.complex(v_real, v_imag), tf.complex64)
    return vrf


def Rate_func(temp):
    h, v, SNR_input = temp
    hv = k.batch_dot(
        tf.cast(h, tf.complex64), tf.transpose(a=v, perm=[1, 0]))
    rate = tf.math.log(tf.cast(1 + SNR_input / Nt * tf.pow(tf.abs(hv), 2), tf.float32)) / tf.math.log(2.0)
    return -rate

In [49]:
imperfect_CSI = Input(name='imperfect_CSI', shape=(H_20input.shape[1:4]), dtype=tf.float32)
# perfect_CSI is only used to compute the loss, and not required in prediction
perfect_CSI = Input(name='perfect_CSI', shape=(H_20.shape[1],), dtype=tf.complex64)
# the SNR is also fed into the BFNN
SNR_input = Input(name='SNR_input', shape=(1,), dtype=tf.float32)
temp = BatchNormalization()(imperfect_CSI)
temp = Flatten()(temp)
temp = BatchNormalization()(temp)
temp = Dense(256, activation='relu')(temp)
temp = BatchNormalization()(temp)
temp = Dense(128, activation='relu')(temp)
phase = Dense(Nt)(temp)
V_RF = Lambda(trans_Vrf, dtype=tf.complex64, output_shape=(Nt,))(phase)
rate = Lambda(Rate_func, dtype=tf.float32, output_shape=(1,))([perfect_CSI, V_RF, SNR_input])
model = tf.keras.Model(inputs=[imperfect_CSI, perfect_CSI, SNR_input], outputs=rate)
# the y_pred is the actual rate, thus the loss is y_pred, without labels
model.compile(optimizer='adam', loss=lambda y_true, y_pred: y_pred)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
imperfect_CSI (InputLayer)      [(None, 1, 2, 64)]   0                                            
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 1, 2, 64)     256         imperfect_CSI[0][0]              
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 128)          0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 128)          512         flatten_5[0][0]                  
_______________________________________________________________________________________

In [ ]:
history = model.evaluate(x=[H_20input, H_20, np.ones([H_20.shape[0], 1]) * -2], y=H_20, batch_size=10000)

In [ ]:
rate = []
for snr in range(-20, 25, 5):
    SNR = np.power(10, np.ones([H.shape[0], 1]) * snr / 10)
    y = model.evaluate(x=[H_input, H, SNR], y=H, batch_size=10000)
    rate.append(-y)
print(rate)

plt.title("The result of BFNN")
plt.xlabel("SNR(dB)")
plt.ylabel("Spectral Efficiency")
plt.plot(range(-20, 25, 5), rate)
plt.show()